In [1]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from churnmodels.schema import Subscription, Event, Account

# the following example will depend on a sqlite db
sqlfile="../churn.db"
engine = create_engine(f"sqlite:///{sqlfile}")
session = sessionmaker(bind=engine)()


In [2]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days)



In [6]:
from sqlalchemy import func, or_
from churnmodels.schema import Account, Subscription, Event, EventType
import pandas as pd

d_start_date = "2020-01-01"
d_end_date = "2020-03-01"

d_start_date = "2020-03-01"
d_end_date = "2020-04-01"

## event_count (§3.1)

In [18]:
d_the_date = "2020-04-01"
the_date = datetime.strptime(d_the_date, "%Y-%m-%d")
inactivity_interval=relativedelta(months=+1)
the_date_int = the_date-inactivity_interval

q_event_count = session.query(Event.account_id, EventType.event_type_name, func.count(Event.account_id).label("n_event2measure_permonth"))\
    .filter(Event.event_time > the_date_int, Event.event_time <= the_date)\
    .join(EventType, EventType.event_type_id==Event.event_type_id)\
    .group_by(Event.account_id, EventType.event_type_name)

event_count = pd.read_sql(q_event_count.statement, engine) #.set_index("account_id")
#print(event_count)

# we do a pivot with pandas
df_pivot = event_count.pivot(index="account_id", columns="event_type_name", values="n_event2measure_permonth")
print(df_pivot)

event_type_name  adview  dislike   like  message  newfriend  post  reply  \
account_id                                                                 
1                  14.0      7.0    8.0     18.0        2.0  17.0   15.0   
2                  16.0     10.0   60.0      1.0        2.0  38.0    NaN   
4                  21.0      NaN   62.0     33.0        8.0  27.0   14.0   
5                   3.0      6.0   21.0     38.0        1.0  44.0   26.0   
6                  21.0      1.0    9.0    165.0        NaN   9.0   22.0   
...                 ...      ...    ...      ...        ...   ...    ...   
12096              26.0      9.0   60.0      3.0        5.0  18.0    2.0   
12097              11.0      8.0  107.0     67.0        1.0  23.0   15.0   
12098              58.0     53.0   50.0    108.0        2.0   7.0   19.0   
12099               1.0     47.0   64.0      NaN       14.0  17.0    NaN   
12100              10.0      4.0    4.0    146.0        NaN   4.0   68.0   

event_type_

## metrics count (§3.2)